![](../img/330-banner.png)

Lecture 4: $k$-Nearest Neighbours and SVM RBFs  
------------
UBC 2022-23 W2

Instructor: Amir Abdi

iclicker link: https://join.iclicker.com/EMMJ <img src="img_aa/iclicker_qr_code.png" height="300" width="300"> 

<!--
replace (img/ wih (../img/  
replace https://join.iclicker.com/3DP5H with https://join.iclicker.com/EMMJ
replace "data/ with "../data
copy legends
copy setup cell
-->

## Announcements

- hw2 deadline is Jan 23, 11:59pm
- [Last day to withdraw without a W standing](https://students.ubc.ca/enrolment/dates-deadlines#2022-23-winter-session-term-2): Jan 23, 2023


<br><br>

### Learning outcomes

From this lecture, you will be able to 

- explain the notion of **similarity-based algorithms**; 
- broadly describe how **$k$-NN** use distances; 
- discuss the effect of using a small/large value of the hyperparameter $k$ when using the $k$-NN algorithm; 
- describe the problem of **curse of dimensionality**; 
- explain the general idea of **SVMs** and **RBF kernel**;
  - broadly describe the relation of `gamma` and `C` hyperparameters of SVMs with the fundamental tradeoff.

<br><br>

### Quick recap

After last session you should know the following; if you don't go back and re-read the content:
- **Train/Valid/Test** datasets and how they help you make better predictions on Prod data.
- **Over-fitting** vs. **Under-fitting**
- The **Bias** vs. **Variance** Tradeoff
- **Golden rule of ML** (not look at test data too often)

<img src="https://miro.medium.com/max/1240/1*feFntGUIiob7MwUX62jdCg.webp" height="500" width="500"> 

Source: https://prvnk10.medium.com/bias-variance-tradeoff-ebf13adcea42


Total Error of the model = $Bias_{error} + Variance_{error} + \sigma$ 
- **Bias** of the learning method: error of simplifying assumptions built into the method (e.g. DecisionTree is a simplifying assumption)
- **Variance** of the learning method: How much the learning method will move around its mean **as the data changes**
- **Irreducible error**: When `X` doesn't fully determine `y`

## Legends


A simple method that everyone seems to have overlooked is to enclose the table within a div and use the align="center" property on it.

    
| <img src="https://upload.wikimedia.org/wikipedia/commons/f/f8/This_is_the_photo_of_Arthur_Samuel.jpg" width="100"> | <img src="http://www.cs.cmu.edu/~tom/TomHead2-6-22-22.jpg" width="100">  | <img src="https://upload.wikimedia.org/wikipedia/commons/4/49/John_McCarthy_Stanford.jpg" width="100"> |
| :-----------: | :-----------: | :-----------: |
| Arthur Samuel       | Tom Mitchell       |John McCarthy|
| (1901-1990)    | 1951 - Now       |  1927 – 2011 |
| First computer learning program | 1997 ML Texbook, CMU Prof | Co-coined term AI, Lisp,<br> Time-sharing, Garbage collection



<br><br>

<br><br>

---------------
If you want to run this notebook you will have to install `ipywidgets`. 
Follow the installation instructions [here](https://ipywidgets.readthedocs.io/en/latest/user_install.html).

---------------

In [ ]:
# Setup

# install ipywidgets if not already installed
!pip install ipywidgets

# Some helper functions to save us time
import sys
sys.path.append("../code/.")

import glob
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
import numpy as np
import mglearn

plt.rcParams["font.size"] = 16
pd.set_option("display.max_colwidth", 200)

# Motivation and distances [[video](https://youtu.be/hCa3EXEUmQk)]

## Similarity (Analogy)-based models

- Suppose you are given the following training examples with corresponding labels and are asked to label a given test example.

![](../img/knn-motivation.png)
<!-- <img src='./img/knn-motivation.png' width="1000"> -->

source of image: https://vipl.ict.ac.cn/en/database.php

- An intuitive way to classify the test example is by finding the most "similar" example(s) from the training set and using that label for the test example.  

### Analogy-based algorithms in practice

- High-tech Facial Recognition
    - Feature vectors for human faces 
    - $k$-NN to identify which face is on their watch list
- Recommendation systems     
- Modern Machine Learning: Embeddings of a Large Language model (e.g. GPT) to find similar documents to a given document

### General idea of $k$-nearest neighbours algorithm

- Consider the following toy dataset with two classes.
    - blue circles $\rightarrow$ class 0
    - red triangles $\rightarrow$ class 1 
    - green stars $\rightarrow$ test examples

In [ ]:
X, y = mglearn.datasets.make_forge()
X_test = np.array([[8.2, 3.66214339], [9.9, 3.2], [11.2, 0.5]])

In [ ]:
from plotting_functions import plot_train_test_points
plot_train_test_points(X, y, X_test)

- Given a new data point, predict the class of the data point by finding the "**closest**" data point in the training set,   
  - Decide the class based on similarity by finding its "**nearest neighbour**" or 
  - **majority vote** of nearest neighbours. 

In [ ]:
from plotting_functions import plot_knn_clf
def f(n_neighbors):
    return plot_knn_clf(X, y, X_test, n_neighbors=n_neighbors)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive
interactive(
    f,
    n_neighbors=widgets.IntSlider(min=1, max=10, step=2, value=1),
)

## Geometric view of features (What is a Dimension?)

- To understand similarity (analogy)-based algorithms it's useful to think of data as points in a high dimensional space. 
- Our `X` represents the problem in terms of relevant **features** ($d$) with one dimension for each **feature** (column).
- Examples are **points in a $d$-dimensional space**. 

How many dimensions (features) are there in the cities data?

In [ ]:
cities_df = pd.read_csv("../data/canada_usa_cities.csv")
X_cities = cities_df[["longitude", "latitude"]]
y_cities = cities_df["country"]

In [ ]:
mglearn.discrete_scatter(X_cities.iloc[:, 0], X_cities.iloc[:, 1], y_cities)
plt.xlabel("longitude")
plt.ylabel("latitude");

In the [Spotify Song Attributes](https://www.kaggle.com/geomack/spotifyclassification/home), how many dimensions does it have?


In [ ]:
spotify_df = pd.read_csv("../data/spotify.csv", index_col=0)
X_spotify = spotify_df.drop(columns=["target", "song_title", "artist"])
X_spotify.head()

In [ ]:
print("The number of features in the Spotify dataset: %d" % X_spotify.shape[1])

The [mnist dataset](https://en.wikipedia.org/wiki/MNIST_database#cite_note-1) is an important (but basic) computer vision dataset
- Characters 0 to 9
- You can develop a solution for MNIST in 5 minutes with 99% accuracy :)
- Each image is **128 x 128** pixels
- **Regression** or **Classification** problem?


<img src="https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png" width="300"> <img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="500">


**Question for you:**
Number of features in MNIST dataset?

### Dimensions in ML problems 

Very naively put (**don't quote me on this**):

- $d \approx 20$ is considered low dimensional
- $d \approx 1000$ is considered medium dimensional 
- $d \approx 100,000$ is considered high dimensional 
  - If you have an image input of `1024 x 768`, that's a 800k dimension feature space.

### Feature vectors 

**Feature vector**
: is composed of feature values associated with an example.

Some example feature vectors are shown below. 

In [ ]:
print(
    "An example feature vector from the Spotify dataset: \n\n%s"
    % (X_spotify.iloc[0].to_numpy())
)

X_spotify.head()

### Similarity between examples

Let's take 2 points (two feature vectors) from the cities dataset.

In [ ]:
two_cities = X_cities.sample(2, random_state=120)
two_cities

The two sampled points are shown as big black circles.

In [ ]:
mglearn.discrete_scatter(
    X_cities.iloc[:, 0], X_cities.iloc[:, 1], y_cities, s=8, alpha=0.3
)
mglearn.discrete_scatter(
    two_cities.iloc[:, 0], two_cities.iloc[:, 1], markers="o", c="k", s=18
);

## Similarity Metrics and Similarity Criterions

### Euclidean distance 

$distance(u, v) = \sqrt{\sum_{i =1}^{n} (u_i - v_i)^2}$  
$u = <u_1, u_2, \dots, u_n>$
$v = <v_1, v_2, \dots, v_n>$

where
- $u$, $v$ are feature vectors of two samples
- $n$ is the dimensionality of data


For the cities, **Euclidean Distance** makes a lot of sense.

For the cities at the two big circles, what is the _distance_ between them?

In [ ]:
two_cities

- Subtract the two cities
- Square the difference
- Sum them up 
- Take the square root 

In [ ]:
# Subtract the two cities
print("Subtract the cities: \n%s\n" % list((two_cities.iloc[1] - two_cities.iloc[0])))

# Squared sum of the difference
print(
    "Sum of squares: %0.4f" % (np.sum((two_cities.iloc[1] - two_cities.iloc[0]) ** 2))
)

# Take the square root
print(
    "Euclidean distance between cities: %0.4f"
    % (np.sqrt(np.sum((two_cities.iloc[1] - two_cities.iloc[0]) ** 2)))
)

In [ ]:
# Euclidean distance using sklearn
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(two_cities)

In [ ]:
import sklearn
sklearn.metrics.pairwise_distances(two_cities, metric='euclidean')

In [ ]:
sklearn.metrics.pairwise_distances(two_cities, metric='l1')


### Other Similarity Metrics
Note: `scikit-learn` supports a number of other [distance metrics](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In the order I expect you to prioritize learning them:
- euclidean (l2)
- manhattan (l1) (cityblock) (sum of absolute distances: ${\sum_{i =1}^{n}|u_i - v_i|}$  
- **cosine**
- mahalanobis
- haversine (angular distance between two points on a surface of a sphere)
- and a few more (chebyshev, minkowski, wminkowski, seuclidean, ...)

Check scikit-learn to learn more:
-  https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html

### Finding the nearest neighbour of City 0

- Let's look at distances from all cities to all other cities

In [ ]:
dists = euclidean_distances(X_cities)
dists.shape

In [ ]:
pd.DataFrame(dists)

In [ ]:
np.fill_diagonal(dists, np.inf)
pd.DataFrame(dists)

Let's look at the distances between **City 0** and some other cities. 

Which city is the closest to **City 0** based on the **Euclidean Distance**?

In [ ]:
# vector
# min
# argmin
# ????

In [ ]:
# Uncomment the following if to learn more details
# print("Feature vector for city 0: \n%s\n" % (X_cities.iloc[0]))
# print("Distances from city 0 to the first 5 cities: %s" % (dists[0][:5]))

# We can find the closest city with `np.argmin`:
print(
    "The closest city from city 0 is: %d \n\nwith feature vector: \n%s"
    % (np.argmin(dists[0]), X_cities.iloc[np.argmin(dists[0])])
)

**Question**

- Would we get the same results if we used a different metric (example, `l1` or `haversine`)?

In [ ]:
# sklearn.metrics.pairwise_distances(X_cities, metric='l1')...
# sklearn.metrics.pairwise_distances(X_cities, metric='haversine')...

### Finding the nearest neighbor of a query point (city)

We can also find the distances to a new "test" or "query" city:

In [ ]:
# Let's find a city that's closest to the a query city
query_point = [[-80, 25]]

dists = euclidean_distances(X_cities, query_point)
print(dists.shape)
# dists[0:10]

In [ ]:
# The query point is closest to
print(
    "The query point %s is closest to the city with index %d and the distance between them is: %0.4f"
    % (query_point, np.argmin(dists), dists[np.argmin(dists)])
)

<br><br>

# $k$-Nearest Neighbours ($k$-NNs) [[video](https://youtu.be/bENDqXKJLmg)]

Evelyn Fix and Joseph Hodges are credited with the initial ideas around the KNN model [in this 1951 paper](https://apps.dtic.mil/sti/pdfs/ADA800276.pdf),  
while Thomas Cover expands on their concept in [his research](https://isl.stanford.edu/~cover/papers/transIT/0021cove.pdf) “Nearest Neighbor Pattern Classification.”
> Reference: https://www.ibm.com/topics/knn

In [ ]:
small_cities = cities_df.sample(30, random_state=90)
one_city = small_cities.sample(1, random_state=44)
small_train_df = pd.concat([small_cities, one_city]).drop_duplicates(keep=False)

In [ ]:
X_small_cities = small_train_df.drop(columns=["country"]).to_numpy()
y_small_cities = small_train_df["country"].to_numpy()
test_point = one_city[["longitude", "latitude"]].to_numpy()

In [ ]:
plot_train_test_points(
    X_small_cities,
    y_small_cities,
    test_point,
    class_names=["Canada", "USA"],
    test_format="circle",
)

- Given a new data point, predict the class of the data point by finding the "closest" data point in the training set, i.e., by finding its "nearest neighbour" or majority vote of nearest neighbours. 

Suppose we want to predict the class of the black point.  
- An intuitive way to do this is predict the same label as the "closest" point ($k = 1$) (1-nearest neighbour)
- We would predict a target of **USA** in this case.

In [ ]:
plot_knn_clf(
    X_small_cities,
    y_small_cities,
    test_point,
    n_neighbors=1,
    class_names=["Canada", "USA"],
    test_format="circle",
)

How about using $k > 1$ to get a more robust estimate? 
- For example, we could also use the 3 closest points (*k* = 3) and let them **vote** on the correct class.  
- The **Canada** class would win in this case. 

In [ ]:
plot_knn_clf(
    X_small_cities,
    y_small_cities,
    test_point,
    n_neighbors=3,
    class_names=["Canada", "USA"],
    test_format="circle",
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k_values = [1, 3, 5]
for k in k_values:

    # ---- New function ------
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_small_cities, y_small_cities)
    # -----------------------
    
    print(
        "Prediction of the black dot with %d neighbours: %s"
        % (k, neigh.predict(test_point))
    )

<br><br><br><br><br><br>
In the KNN algorithm, `k` matters!

is `k` a **hyper-parameter** or **parameter**?
<br><br><br><br><br><br>

## Choosing `n_neighbors`

- The primary hyperparameter of the model is `n_neighbors` ($k$) which decides how many neighbours should vote during prediction? 
- What happens when we play around with `n_neighbors`?
- Are we more likely to **overfit** with a low `n_neighbors` or a high `n_neighbors`?
- Let's examine the effect of the hyperparameter on our cities data. 

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate

X = cities_df.drop(columns=["country"])
y = cities_df["country"]

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.9, random_state=123
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, train_size=0.8, random_state=123
)

In [ ]:
k = 1
knn1 = KNeighborsClassifier(n_neighbors=k)
score = knn1.fit(X_train, y_train).score(X_valid, y_valid)
print('Validation Accuracy:', score)

In [ ]:
k = 10
knn100 = KNeighborsClassifier(n_neighbors=k)
score = knn100.fit(X_train, y_train).score(X_valid, y_valid)
print('Validation Accuracy:', score)

In [ ]:
k = 20
knn100 = KNeighborsClassifier(n_neighbors=k)
score = knn100.fit(X_train, y_train).score(X_valid, y_valid)
print('Validation Accuracy:', score)

In [ ]:
# For students: Play around with this for fun!
# ------------- To be skipped ----------------
def f(n_neighbors=1):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    score_valid = knn.score(X_valid, y_valid)
    score_train = knn.score(X_train, y_train)
    print('n_neighbors:', n_neighbors)
    print('train score:', score_train)
    print('valid score:', score_valid)

interactive(
    f,
    n_neighbors=widgets.IntSlider(min=1, max=101, step=5, value=1),
)
# ------------- To be skipped ----------------

In [ ]:
import matplotlib
from plotting_functions import plot_knn_decision_boundaries
matplotlib.rc('font', **{'size'   : 10})

plot_knn_decision_boundaries(X_train, y_train, k_values=[1, 10, 20])

In [ ]:
scores_valid, scores_train = list(), list()
n_neighbors_list = range(1, 50, 5)
for n_neighbors in n_neighbors_list:
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    scores_valid.append(knn.score(X_valid, y_valid))
    scores_train.append(knn.score(X_train, y_train))

results_df = pd.DataFrame({'train_accuracy': scores_train, 'valid_accuracy': scores_valid, 'n_neighbors': n_neighbors_list}).set_index('n_neighbors')
results_df.plot();

<br><br><br><br>
**Question**:
Why at `n_neighbors=1`, Train Accuracy is 100%?
<br><br><br><br><br><br><br><br><br><br>

## What's the performance of the best model on test set?

- `n_neighbors` is a hyperparameter
- We can use hyperparameter optimization to choose `n_neighbors`.

In [ ]:
results_df

In [ ]:
results_df.idxmax()

**Question**: Do you notice some weird pattern above? How do you explain that?

------------------
Now, we will evaluate our best model on **test data** to report to our stakeholders (customer, product manager, marketting, etc.):

In [ ]:
pd.concat((y_train, y_valid))

In [ ]:
best_n_neighbours = results_df.idxmax()['valid_accuracy']
knn = KNeighborsClassifier(n_neighbors=best_n_neighbours)

# Let's use the entire Train + Valid dataset to train a new model
knn.fit(pd.concat((X_train, X_valid)), pd.concat((y_train, y_valid)))

print("Test accuracy: %0.3f" % (knn.score(X_test, y_test)))

<br><br>

## ❓❓ Questions for you 

### (iClicker) Exercise 4.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

1. Similarity (analogy)-based models find examples from the test set that are most similar to the query example we are predicting.
2. We more likely to **overfit** with a low `n_neighbors` than a high `n_neighbors`
3. With $k$-NN, setting the hyperparameter $k$ to larger values typically reduces training error. 
4. In $k$-NN, with $k > 1$, the classification of the closest neighbour to the test example always contributes the most to the prediction.
5. KNN is such an old algorithm, oh dear! Why am I learning this? :)

## Break (5 min)

![](../img/eva-coffee.png)


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
```python
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
```
What's KNN learning when we call the `.fit()` function?
<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

- **Lazy learning**: Takes no time to `fit`, i.e, **no fitting!** 
- There is no learning happening
- Has no learning process
- model = all the training data

Compuational complexity of prediction on a new sample? 

- Have to **iterate** over all the sample one-by-one and calculate the **distance** to all, and get the **top k** samples.
- Can be potentially be **VERY slow** during prediction time, especially when the training set is very large.

<br><br><br><br><br><br><br><br><br><br>

## [Study on your own!]
### More on $k$-NNs [[video](https://youtu.be/IRGbqi5S9gQ)]

### Other useful arguments of `KNeighborsClassifier`

- `weights` $\rightarrow$ When predicting label, **you can assign higher weight to the examples which are closer to the query example**.  
- **Exercise for you**: Play around with this argument. Do you get a better validation score? 

### Regression with $k$-nearest neighbours ($k$-NNs)

- Can we solve regression problems with $k$-nearest neighbours algorithm? 
- In $k$-NN regression we take the average of the $k$-nearest neighbours. 
- We can also have **weighted regression**. 

See an example of regression in the lecture notes. 

In [ ]:
mglearn.plots.plot_knn_regression(n_neighbors=1)

In [ ]:
mglearn.plots.plot_knn_regression(n_neighbors=3)

### Pros of $k$-NNs for supervised learning

- Easy to understand, interpret.
- Simple hyperparameter $k$ (`n_neighbors`) controlling the fundamental tradeoff.
- Can learn very complex functions given enough data.

### Cons of $k$-NNs for supervised learning

- Can be potentially be VERY slow during prediction time, especially when the training set is very large. 
- Often not that great test accuracy compared to the modern approaches.
- It does not work well on datasets with many features or where most feature values are 0 most of the time (sparse datasets).    


For regular $k$-NN for supervised learning (not with sparse matrices), you should normalize (scale) your features.   
We'll be looking into it soon. 


# Parametric vs Non-parametric Models

- **Parametric Model:** Learning **finite number of learnable parameters** during training.
  - Eventually, as we add more data, we get to a point where more data doesn’t help (too much bias, model is too simple).
- **Non-parametric Model:** The number of **parameters can be potentially infinite**.

A simplied explanation of **non-parametric model**: The model (what is stored with the model) **increases with the size of data**



**Non-parametric example**: $k$-NN is a classic example of non-parametric models.     

**Question**: Are **DecisionTrees** parametric or non-parametric?

Answer: ???



- If you want to know more about this, find some reading material 
  - https://www.cs.ubc.ca/~schmidtm/Courses/340-F16/L6.pdf
  - http://mlss.tuebingen.mpg.de/2015/slides/ghahramani/gp-neural-nets15.pdf
  - https://machinelearningmastery.com/parametric-and-nonparametric-machine-learning-algorithms/
- By the way, the terms "parametric" and "non-paramteric" are often used differently by statisticians, see [here](https://help.xlstat.com/s/article/what-is-the-difference-between-a-parametric-and-a-nonparametric-test?language=en_US) for more...

<br><br><br><br><br><br><br><br>
# Curse of dimensionality

**When more features HARM rather than HELP**

- Affects all learners but especially bad for similarity-based models. 

## Curse of Dimensionality on KNN

- $k$-NN usually works well when the number of dimensions $d$ is small but things fall apart quickly as $d$ goes up.
  - **Because $k$-NN cannot ignore any features**
- If there are many irrelevant attributes, $k$-NN is hopelessly confused because all of them contribute to finding similarity between examples. 
- With enough irrelevant attributes the accidental similarity swamps out meaningful similarity and $k$-NN is no better than random guessing.  

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=30, n_features=4, n_classes=2)
X

```python
from sklearn.dummy import DummyClassifier
```
Randomly selects one of the two classes; it's expected accuracy is 50% on binary classification

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.datasets import make_classification

nfeats_accuracy = {"nfeats": [], "dummy_valid_accuracy": [], "KNN_valid_accuracy": []}
for n_feats in range(4, 2000, 100):
    X, y = make_classification(n_samples=2000, n_features=n_feats, n_classes=2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

    # DummyClassifier 
    dummy = DummyClassifier(strategy="most_frequent")
    dummy_scores = cross_validate(dummy, X_train, y_train, return_train_score=True)

    # KNeighborsClassifier 
    knn = KNeighborsClassifier()
    scores = cross_validate(knn, X_train, y_train, return_train_score=True)
    
    nfeats_accuracy["nfeats"].append(n_feats)
    nfeats_accuracy["KNN_valid_accuracy"].append(np.mean(scores["test_score"]))
    nfeats_accuracy["dummy_valid_accuracy"].append(np.mean(dummy_scores["test_score"]))

In [ ]:
pd.DataFrame(nfeats_accuracy)

<br><br>

# Support Vector Machines (SVMs) with RBF kernel [[video](https://youtu.be/ic_zqOhi020)]


<img src="https://datascience.columbia.edu/wp-content/uploads/2020/08/Vapnik_web.png" width="300">

Vladimir N. Vapnik, 1936 (age 86)

> The original SVM algorithm was invented by **Vladimir N. Vapnik** and Alexey Ya. Chervonenkis** in 1963.   
In 1992, **Bernhard Boser, Isabelle Guyon and Vladimir Vapnik** suggested a way to create nonlinear classifiers by applying the kernel trick to maximum-margin hyperplanes.

- Very high-level overview
- Our goals here are
    - Use `scikit-learn`'s SVM model. 
    - Broadly explain the notion of support vectors.  
    - Broadly explain the similarities and differences between $k$-NNs and SVM RBFs.
    - Explain how `C` and `gamma` hyperparameters control the fundamental tradeoff.
    
> (Optional) RBF stands for radial basis functions. We won't go into what it means in this session. Refer to [this video](https://www.youtube.com/watch?v=Qc5IyLW_hns) if you want to know more. 

## What is a Support Vector?

Samples on the **boundaries** that determine the **margin hyperplane**


A **hyperplane** can be defined as: 

${\displaystyle \mathbf {w} ^{\mathsf {T}}\mathbf {x} -b=0}$

$w$ and $b$ are learnable parameters, and learned during training of SVM
<br><br><br><br><br>

This is is a **Linear HARD SVM** 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/SVM_margin.png/1920px-SVM_margin.png" width="500">

[img source](https://en.wikipedia.org/wiki/Support_vector_machine)

**HARD**: No misclassification can happen  
**SOFT**: Misclassification can happen, and each miscalssification will be penalized relative to `C`


Check this interactive demo: https://greitemann.dev/svm-demo

## Overview

- Another popular similarity-based algorithm is Support Vector Machines with RBF Kernel (SVM RBFs)
- Superficially, SVM RBFs are more like weighted $k$-NNs.
    - The decision boundary is defined by **a set of positive and negative examples** and **their weights** together with **their similarity measure**. 

- The primary difference between $k$-NNs and SVM RBFs is that 
    - Unlike $k$-NNs, SVM RBFs only remember the key examples (**support vectors**). 
    - SVMs use a different similarity metric which is called a "kernel". 
      - A popular kernel is Radial Basis Functions (RBFs)
    - They usually perform better than $k$-NNs

Check this interactive demo: https://greitemann.dev/svm-demo

## Let's explore SVM RBFs

Let's try SVMs on the cities dataset. 

In [ ]:
mglearn.discrete_scatter(X_cities.iloc[:, 0], X_cities.iloc[:, 1], y_cities)
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.legend(loc=1);

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_cities, y_cities, test_size=0.2, random_state=123
)

<br><br><br><br><br><br><br><br>
An SVM used for **C**lassificatio is called **SVC**  
Simiilarly,  
An SVM used for **R**egression is called **SVR**  
<br><br><br><br><br><br><br><br>

In [ ]:
from sklearn.svm import SVC

# New Class: SVC!
svm = SVC(gamma=0.01)  # Ignore gamma for now
scores = cross_validate(svm, X_train, y_train, return_train_score=True)

print("Mean validation score %0.3f" % (np.mean(scores["test_score"])))
pd.DataFrame(scores)

### Decision boundary of SVMs 
- We can think of SVM with RBF kernel as "smooth KNN". 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

for clf, ax in zip([knn, svm], axes):
    clf.fit(X_train, y_train)
    mglearn.plots.plot_2d_separator(
        clf, X_train.to_numpy(), fill=True, eps=0.5, ax=ax, alpha=0.4
    )
    mglearn.discrete_scatter(X_train.iloc[:, 0], X_train.iloc[:, 1], y_train, ax=ax)
    ax.set_title(clf)
    ax.set_xlabel("longitude")
    ax.set_ylabel("latitude")

### Support vectors 

- Each training example either is or isn't a "support vector".
  - This gets decided during `fit`.

- **Main insight: the decision boundary only depends on the support vectors.**

- Let's look at the support vectors. 

In [ ]:
from sklearn.datasets import make_blobs

n = 20
n_classes = 2
X_toy, y_toy = make_blobs(
    n_samples=n, centers=n_classes, random_state=300
)  # Let's generate some fake data

In [ ]:
mglearn.discrete_scatter(X_toy[:, 0], X_toy[:, 1], y_toy)
plt.xlabel("Feature 0")
plt.ylabel("Feature 1");

In [ ]:
mglearn.discrete_scatter(X_toy[:, 0], X_toy[:, 1], y_toy)
plt.xlabel("Feature 0")
plt.ylabel("Feature 1");

svm = SVC(kernel="rbf", C=10, gamma=0.1).fit(X_toy, y_toy)
mglearn.plots.plot_2d_separator(svm, X_toy, fill=True, eps=0.5, alpha=0.4)

In [ ]:
svm.support_

In [ ]:
from plotting_functions import plot_support_vectors
plot_support_vectors(svm, X_toy, y_toy)

The support vectors are the bigger points in the plot above. 

### Hyperparameters of SVM 

- Key hyperparameters of **RBF (Radial Basis Function)** SVM are
    - `gamma`: Defines **how far** the influence of a single training example reaches
    - `C`: The C parameter **trades off correct classification** vs. **maximization of the decision margin**
    
- We are not equipped to understand the meaning of these parameters at this point but you are expected to describe their relation to the fundamental tradeoff. 

See [`scikit-learn`'s explanation of RBF SVM parameters](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html). 

### Relation of `gamma` and the fundamental trade-off

- `gamma` controls the complexity (fundamental trade-off), just like other hyperparameters we've seen.
  - larger `gamma` $\rightarrow$ each point influences close, **more complex**
  - smaller `gamma` $\rightarrow$ each point influence far away, **less complex**

In [ ]:
from plotting_functions import plot_svc_gamma
matplotlib.rc('font', **{'size'   : 10})

gamma = [0.001, 0.01, 0.1, 1.0, 2.0]
plot_svc_gamma(
    gamma,
    X_train.to_numpy(),
    y_train.to_numpy(),
    x_label="longitude",
    y_label="latitude",
)

### Relation of `C` and the fundamental trade-off

- `C` _also_ affects the fundamental tradeoff
- The bigger the `C` the more you **penalize misclssification**
    - larger `C` $\rightarrow$ penalize more, **more complex**, smaller decision margin
    - smaller `C` $\rightarrow$ penalize less, **less complex**, bigger decision margin
    

<br><br>

### $nu$-SVM [Optional/Bonus Content]
There is another version of SVM calld **$\nu$-SVM** where `C` is replaced by `nu`
- `nu` ranges from 0 to 1
- `C` ranges from 0 to infinity
- **Larger values of `nu`** correspond to **smaller values of `C`**
- **Smaller values of `nu`** correspond to **larger values of `C`**
- `C = infinity` $\rightarrow$ `nu = 0` $\rightarrow$ HARDer margin
- `C = 0` $\rightarrow$ `nu = 1` $\rightarrow$ SOFTer margin


In [ ]:
from plotting_functions import plot_svc_C
matplotlib.rc('font', **{'size'   : 10})

C = [0.1, 1.0, 10.0, 100.0, 1000.0]
plot_svc_C(
    C, X_train.to_numpy(), y_train.to_numpy(), x_label="longitude", y_label="latitude"
)

### SVM Regressor

- Similar to KNNs, you can use SVMs for regression problems as well.
- See [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) for more details.

# HParam Search over multiple hyperparameters
What to do when you have 2+ HParam to decide?

- In the above case the best training error is achieved by the most complex model (large `gamma`, large `C`).
- Best validation error requires a hyperparameter search to balance the fundamental tradeoff.
- How do you check all values of all Hyper-Parameters to decide the **best set of HParams?**

- More on this next week. But if you cannot wait till then, you may look up the following:
    - [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
    - [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

## ❓❓ Questions for you 

## (iClicker) Exercise 4.2 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

1. $k$-NN may perform poorly in high-dimensional space (say, *d* > 1000). 
2. In SVM RBF, removing a non-support vector would not change the decision boundary. 
3. In sklearn’s SVC classifier, large values of gamma tend to result in higher training score but probably lower validation score. 
4. If we increase both gamma and C, we can't be certain if the model becomes more complex or less complex.

#### More practice questions 

- Check out some more practice questions [here](https://ml-learn.mds.ubc.ca/en/module4).

## Summary

- We have **KNNs and SVMs as supervised learning** techniques in our toolbox.
- These are similarity (analogy)-based learners and the idea is to assign nearby points the same label.
- **Unlike decision trees, all features are equally important**
- Both can be used for **classification** or **regression** (much like the other methods we've seen).

### Coming up:

Lingering questions: 
- Are we ready to do machine learning on real-world datasets?
- What would happen if we use $k$-NNs or SVM RBFs on the spotify dataset from hw1?  
- What happens if we have missing values in our data? 
- What do we do if we have features with categories or string values?


![](../img/eva-seeyou.png)